### purpose: Incoperate MAF for wightman so i can calculate beta score for wightman 
(Previously, I used p instead of MAF, which is wrong. So I reran it)
### update on 0928 
I found there are actually some duplicated SNPs which is weird. So I did a lot more analysis thats shown belowed


In [2]:
import pandas as pd
import numpy as np

In [3]:
wightman = pd.read_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/Wightman_et_al_2021_hg37_ldsc.tsv.gz', compression='gzip',sep='\t')


In [14]:
#jansen = pd.read_parquet('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/processed/jansen_et_al_2021_hg37_ldsc.munged.parquet')
#jansen_og = pd.read_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/Jansen_et_al_2019_hg37_ldsc.tsv.gz', compression = 'gzip', sep = '\t')

### MAF was opposite while A1 and A2 are still the same

In [15]:
print(jansen[jansen.SNP == 'rs12184277'])
print(jansen_og[jansen_og.SNP == 'rs12184277'])


          SNP  CHR      BP A1 A2         Z       MAF       N
1  rs12184277    1  715367  A  G -1.957915  0.041069  360230
          SNP  CHR      BP A1 A2   UNIQID.A1A2         Z        P    NSUM  \
1  rs12184277    1  715367  A  G  1:715367_G_A -1.957915  0.05024  360230   

       NEFF   DIR       FRQ      BETA        SE       N  
1  360230.0  ??+?  0.958931 -0.011624  0.005937  360230  


In [8]:
0.9591931+0.0408


0.9999931

In [3]:
wightman

,SNP,CHR,BP,A1,A2,Z,P,N
0,rs533499096,1,14860,G,T,-1.275874,0.202000,74004
1,rs373918278,1,17765,G,A,0.093886,0.925200,74004
2,rs763779009,1,17767,G,A,-0.426148,0.670000,74004
3,rs199900651,1,48186,T,G,0.913081,0.361200,74004
4,rs555296411,1,108506,C,T,0.269688,0.787400,74004
...,...,...,...,...,...,...,...,...
11991345,rs149733995,22,51238249,A,C,0.159780,0.873054,137930
11991346,rs574579322,22,51239584,G,C,-0.768483,0.442200,74004
11991347,rs535432390,22,51239586,T,G,-0.737201,0.461000,74004
11991348,rs3888401,22,51242557,A,T,-1.158893,0.246500,74004


### only need to merge MAF for wightman

In [4]:
wightman_maf = pd.DataFrame()
for chrom in range(1,23):
    print("start chr %d"%(chrom))
    maf_path = '/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/ukb_maf/ukb_mfi_chr' + str(chrom) +"_v3.txt"
    maf_info = pd.read_csv(maf_path, sep = '\t', names = ['UNIQID.A1A2', 'SNP','BP','A1','A2','MAF','NA','INFO'], usecols= ['SNP','BP','A1','A2','MAF'])
    MAF=wightman[wightman.CHR == chrom].merge(maf_info, how = 'inner', on='SNP')
    wightman_maf = pd.concat([wightman_maf, MAF], ignore_index = True)


start chr 1
start chr 2
start chr 3
start chr 4
start chr 5
start chr 6
start chr 7
start chr 8
start chr 9
start chr 10
start chr 11
start chr 12
start chr 13
start chr 14
start chr 15
start chr 16
start chr 17
start chr 18
start chr 19
start chr 20
start chr 21
start chr 22


### deal with duplicated RSID first

In [5]:
## There are some duplicated rsid
duplicated_SNP = wightman_maf[wightman_maf.SNP.duplicated()].SNP
duplicated_SNP_df = wightman_maf[wightman_maf.SNP.isin(duplicated_SNP)]
duplicated_SNP_df

,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF
65,rs151120166,1,715142,G,T,0.487929,0.625600,74004,715142,G,A,0.000198
66,rs151120166,1,715142,G,T,0.487929,0.625600,74004,715142,G,T,0.000194
939,rs529215219,1,932249,C,A,0.087090,0.930600,74004,932249,C,A,0.000060
940,rs529215219,1,932249,C,A,0.087090,0.930600,74004,932249,C,T,0.000733
1470,rs182887483,1,1034498,C,G,0.632898,0.526800,74004,1034498,C,G,0.001859
...,...,...,...,...,...,...,...,...,...,...,...,...
11724039,rs143229588,22,50912058,G,C,-0.694929,0.487100,74004,50912058,G,C,0.000329
11724158,rs558655639,22,50934723,A,C,1.395716,0.162800,74004,50934723,A,C,0.000136
11724159,rs558655639,22,50934723,A,C,1.395716,0.162800,74004,50934723,A,G,0.000081
11725332,rs4824145,22,51144305,G,A,1.338683,0.180674,87106,51144305,G,A,0.378798


In [6]:
## remove the rows that A2 are different ( in the duplicated ones)
inconsistent_a2_df = duplicated_SNP_df[duplicated_SNP_df.apply(lambda x: x['A2_x'] != x['A2_y'], axis = 1)]
inconsistent_a2_df

,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF
65,rs151120166,1,715142,G,T,0.487929,0.625600,74004,715142,G,A,0.000198
940,rs529215219,1,932249,C,A,0.087090,0.930600,74004,932249,C,T,0.000733
1471,rs182887483,1,1034498,C,G,0.632898,0.526800,74004,1034498,C,T,0.000124
1741,rs148495683,1,1084093,G,A,-0.544932,0.585800,74004,1084093,G,T,0.000571
4118,rs140287013,1,1526149,G,A,2.146539,0.031830,63926,1526149,G,T,0.000020
...,...,...,...,...,...,...,...,...,...,...,...,...
11720957,rs75868168,22,50327982,T,A,0.176120,0.860200,74004,50327982,T,C,0.002071
11722427,rs78889708,22,50614596,G,A,0.375058,0.707617,437398,50614596,G,T,0.000033
11724038,rs143229588,22,50912058,G,C,-0.694929,0.487100,74004,50912058,G,A,0.000102
11724159,rs558655639,22,50934723,A,C,1.395716,0.162800,74004,50934723,A,G,0.000081


In [7]:
wightman_remove_inconsistentSNP = wightman_maf[~ wightman_maf.isin(inconsistent_a2_df)].dropna(how = 'all')
print(wightman_remove_inconsistentSNP.shape)
wightman_remove_inconsistentSNP[wightman_remove_inconsistentSNP.SNP.duplicated()] ## theres no duplicated RSID now!

(11715943, 12)


,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF


### Deal with A1 and A2 flipping

In [8]:
wightman_remove_inconsistentSNP[wightman_remove_inconsistentSNP.A1_x == wightman_remove_inconsistentSNP.A2_y] ## some SNP has A1 and A2 flipped. But not the other way around. In this case, I should change their MAF to 1 - MAF
wightman_remove_inconsistentSNP[(wightman_remove_inconsistentSNP.A1_x == wightman_remove_inconsistentSNP.A2_y) & (wightman_remove_inconsistentSNP.A2_x != wightman_remove_inconsistentSNP.A1_y)]  ## none


,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF


In [9]:
## reverse MAF 
wightman_remove_inconsistentSNP.loc[wightman_remove_inconsistentSNP.A1_x == wightman_remove_inconsistentSNP.A2_y,"MAF"] = 1 - wightman_remove_inconsistentSNP[wightman_remove_inconsistentSNP.A1_x == wightman_remove_inconsistentSNP.A2_y].MAF

### Remove other random SNP (i have no idea why they are sharing the same RSID)

In [11]:
## the snps that very different A1 and A2 with the ukbb one, should be removed ## 12 of them in total
## and the BP are not even the same (i have no idea why they are sharing the same RSID)
## add two condition just to be sure that non of the A1, A2 flipping SNPs were included
snp_to_remove = wightman_remove_inconsistentSNP[(wightman_remove_inconsistentSNP.A1_x != wightman_remove_inconsistentSNP.A1_y) & (wightman_remove_inconsistentSNP.A1_x != wightman_remove_inconsistentSNP.A2_y)]
snp_to_remove

,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF
334755,rs12715032,1.0,83638806.0,C,T,-0.391453,0.695462,252501.0,83638807.0,G,A,0.000689
970740,rs12466398,2.0,11992128.0,A,C,-0.425413,0.670536,721953.0,11992129.0,G,C,0.136106
1401223,rs36171435,2.0,118678541.0,T,C,-0.191691,0.847984,762312.0,118678542.0,G,A,0.028166
2282412,rs2873636,3.0,83674829.0,T,C,0.930137,0.352300,573237.0,83674830.0,G,A,0.433542
3535798,rs2644004,4.0,176641241.0,T,C,0.994648,0.319907,760887.0,176641242.0,G,A,0.452554
3699042,rs1849841,5.0,17759236.0,A,T,-0.775025,0.438325,334818.0,17759237.0,T,G,0.260913
4715521,rs77827577,6.0,78313189.0,T,C,-0.812252,0.416647,640207.0,78312719.0,A,G,0.000208
5417317,rs71561073,7.0,63177930.0,T,C,0.822800,0.410622,700997.0,63177931.0,G,A,0.004634
6648288,rs71525264,9.0,33613912.0,C,G,1.407843,0.159178,137930.0,33613913.0,T,G,0.380110
9069175,rs9300476,13.0,98954148.0,T,C,0.881687,0.377946,762917.0,98954149.0,G,A,0.127984


In [12]:
wightman_filtered = wightman_remove_inconsistentSNP[~wightman_remove_inconsistentSNP.isin(snp_to_remove)]
wightman_filtered

,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF
0,rs533499096,1.0,14860.0,G,T,-1.275874,0.2020,74004.0,14860.0,G,A,0.000006
1,rs555296411,1.0,108506.0,C,T,0.269688,0.7874,74004.0,108506.0,C,T,0.005777
2,rs62642118,1.0,108929.0,C,G,0.186567,0.8520,74004.0,108929.0,C,G,0.008015
3,rs372054918,1.0,109503.0,G,A,-0.898723,0.3688,74004.0,109503.0,G,A,0.001644
4,rs528106901,1.0,133855.0,C,T,-0.646895,0.5177,74004.0,133855.0,C,T,0.000311
...,...,...,...,...,...,...,...,...,...,...,...,...
11725639,rs200189535,22.0,51235959.0,T,C,-0.060823,0.9515,74004.0,51235959.0,T,C,0.196421
11725640,rs534679861,22.0,51236137.0,C,T,0.741814,0.4582,74004.0,51236137.0,C,T,0.002108
11725641,rs3896457,22.0,51237063.0,T,C,-0.636884,0.5242,137930.0,51237063.0,T,C,0.292748
11725642,rs574579322,22.0,51239584.0,G,C,-0.768483,0.4422,74004.0,51239584.0,G,C,0.000015


## But there are still some SNPs with the different A2 but shares the same BP and RSID. However, they can't share the same MAF

In [13]:
snp_to_remove = wightman_filtered[(wightman_filtered.A1_x == wightman_filtered.A1_y) & (wightman_filtered.A2_x != wightman_filtered.A2_y)].dropna(how = 'all')
print(snp_to_remove)
wightman_filtered = wightman_filtered[~ wightman_filtered.isin(snp_to_remove)].dropna(how = 'all')
print('%d snps remain'%(wightman_filtered.shape[0]))

                  SNP   CHR        BP_x A1_x A2_x         Z         P  \
0         rs533499096   1.0     14860.0    G    T -1.275874  0.202000   
2385      rs563456062   1.0   1183595.0    G    C  0.022938  0.981700   
4051        rs9442407   1.0   1516270.0    C    G  1.150592  0.249900   
8921        rs6678872   1.0   2626797.0    C    T -0.101833  0.918889   
11265     rs199691852   1.0   3098386.0    G    A  0.629146  0.529254   
...               ...   ...         ...  ...  ...       ...       ...   
11691250  rs777822362  22.0  45259152.0    G    A  0.017923  0.985700   
11703107  rs187009824  22.0  47548526.0    C    G  1.651679  0.098600   
11709572  rs576971736  22.0  48737596.0    G    T  0.947469  0.343400   
11710819    rs7354789  22.0  48908037.0    T    G  0.054923  0.956200   
11720763  rs539151546  22.0  50305741.0    C    G  0.419022  0.675200   

                N        BP_y A1_y A2_y       MAF  
0         74004.0     14860.0    G    A  0.000006  
2385      74004.0  

In [14]:
# check if there are other inconsistency except for the allele flip ones. 
wightman_filtered[(wightman_filtered.A1_x!= wightman_filtered.A1_y)&(wightman_filtered.A2_x!= wightman_filtered.A2_y) & (wightman_filtered.A1_x!= wightman_filtered.A2_y)]

,SNP,CHR,BP_x,A1_x,A2_x,Z,P,N,BP_y,A1_y,A2_y,MAF


## convert to desired DF structure. 

In [16]:
## convert float to int
wightman_filtered[['CHR', 'N','BP_x']] = wightman_filtered[['CHR', 'N','BP_x']].apply(np.int64)

In [17]:
wightman_filtered = wightman_filtered.drop(columns=["A1_y", "A2_y", "BP_y"]).rename(columns= {'A1_x':'A1', 'A2_x':'A2', 'BP_x':'BP'})
wightman_filtered

,SNP,CHR,BP,A1,A2,Z,P,N,MAF
1,rs555296411,1,108506,C,T,0.269688,0.7874,74004,0.005777
2,rs62642118,1,108929,C,G,0.186567,0.8520,74004,0.008015
3,rs372054918,1,109503,G,A,-0.898723,0.3688,74004,0.001644
4,rs528106901,1,133855,C,T,-0.646895,0.5177,74004,0.000311
5,rs554762511,1,135195,A,G,-0.035476,0.9717,74004,0.007480
...,...,...,...,...,...,...,...,...,...
11725639,rs200189535,22,51235959,T,C,-0.060823,0.9515,74004,0.196421
11725640,rs534679861,22,51236137,C,T,0.741814,0.4582,74004,0.002108
11725641,rs3896457,22,51237063,T,C,-0.636884,0.5242,137930,0.292748
11725642,rs574579322,22,51239584,G,C,-0.768483,0.4422,74004,0.000015


## calculate beta for wightman
#### formula: Beta = z / sqrt(2MAF(1− MAF)(n + z^2))

In [18]:
wightman_filtered["BETA"] = wightman_filtered.Z/np.sqrt(2*wightman_filtered.MAF*(1-wightman_filtered.MAF)*wightman_filtered.N +wightman_filtered.Z ** 2 )

In [19]:
wightman_filtered

,SNP,CHR,BP,A1,A2,Z,P,N,MAF,BETA
1,rs555296411,1,108506,C,T,0.269688,0.7874,74004,0.005777,0.009249
2,rs62642118,1,108929,C,G,0.186567,0.8520,74004,0.008015,0.005439
3,rs372054918,1,109503,G,A,-0.898723,0.3688,74004,0.001644,-0.057563
4,rs528106901,1,133855,C,T,-0.646895,0.5177,74004,0.000311,-0.095006
5,rs554762511,1,135195,A,G,-0.035476,0.9717,74004,0.007480,-0.001070
...,...,...,...,...,...,...,...,...,...,...
11725639,rs200189535,22,51235959,T,C,-0.060823,0.9515,74004,0.196421,-0.000398
11725640,rs534679861,22,51236137,C,T,0.741814,0.4582,74004,0.002108,0.042007
11725641,rs3896457,22,51237063,T,C,-0.636884,0.5242,137930,0.292748,-0.002665
11725642,rs574579322,22,51239584,G,C,-0.768483,0.4422,74004,0.000015,-0.452513


In [21]:
wightman_filtered.to_csv('/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/summary_stats/alzheimers/fixed_alzheimers/processed/wightman_fixed_beta.tsv',sep = '\t',index = False)